# Setup

In [8]:
! pip install -qqU transformers torch torchvision torchaudio

zsh:1: command not found: pip


python(12933) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import warnings
import os
warnings.simplefilter(action='ignore', category=FutureWarning)

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

/Users/marcinpilarczyk/projects/python/AI-Notes/_personal/course/gen_ai/day_3/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class CFG:
    model = "microsoft/Phi-3-mini-4k-instruct"
    device = "cpu"

if torch.backends.mps.is_available():
    CFG.device = "mps"
if torch.cuda.is_available():
    CFG.device = "cuda"


print("Device: ", CFG.device)

Device:  mps


# Funkcje

In [ ]:
model = AutoModelForCausalLM.from_pretrained(CFG.model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:50<00:00, 25.39s/it]


RuntimeError: MPS backend out of memory (MPS allocated: 17.96 GB, other allocations: 752.00 KB, max allowed: 18.13 GB). Tried to allocate 192.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [5]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)

In [6]:
def generate_text(
    prompt,
    temperature,
    top_p,
    model=model,
    tokenizer=tokenizer,
    max_length=100,
    num_return_sequences=1,
):
    # Set up the text generation pipeline
    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map=CFG.device,
    )

    # Generate text
    generated = generator(
        prompt,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        temperature=temperature,
        top_p=top_p,
        do_sample=True,
        truncation=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    return generated[0]["generated_text"]

Na początku funkcja przyjmuje szereg parametrów, które kontrolują proces generowania tekstu:
- `prompt` to tekst wejściowy, od którego model zacznie generację
- `temperature` kontroluje losowość generacji - wyższa wartość daje bardziej kreatywne, ale mniej przewidywalne wyniki
- `top_p` określa próg prawdopodobieństwa dla wyboru następnego słowa (nazywany próbkowaniem nucleus)
- `model_name` to nazwa modelu, którego chcemy użyć
- `max_length` ustala maksymalną długość generowanego tekstu (domyślnie 100 tokenów)
- `num_return_sequences` określa liczbę różnych wersji tekstu do wygenerowania (domyślnie 1)

W pierwszym kroku funkcja tworzy obiekt `generator` używając funkcji `pipeline`. Ten generator jest konfigurowany następującymi parametrami:
- typ zadania ustawiony na 'text-generation'
- wskazany model językowy
- `torch_dtype=torch.float16` oznacza użycie 16-bitowej precyzji liczb zmiennoprzecinkowych, co oszczędza pamięć
- `device_map` wskazuje na urządzenie obliczeniowe zdefiniowane w klasie CFG

Następnie funkcja wywołuje generator z wcześniej ustalonymi parametrami. Dodatkowe parametry w tym wywołaniu to:
- `do_sample=True` włącza próbkowanie losowe zamiast wybierania zawsze najbardziej prawdopodobnego słowa
- `truncation=True` pozwala na przycięcie tekstu do zadanej długości
- `pad_token_id=None` wyłącza automatyczne dopełnianie sekwencji

Na końcu funkcja zwraca wygenerowany tekst, wybierając pierwszy element z listy wyników (indeks [0]) i pobierając z niego klucz 'generated_text'.

# Test

In [7]:
prompt = "In a world where technology and nature coexist,"

Te dwa parametry, temperatura i top_p, są kluczowe w procesie generacji tekstu:
- Temperatura działa jak "pokrętło kreatywności". Przy niskich wartościach model będzie bardziej konserwatywny i przewidywalny, wybierając najbardziej prawdopodobne słowa. Przy wysokich wartościach staje się bardziej twórczy i nieprzewidywalny.
- Top_p kontroluje różnorodność słownictwa poprzez ograniczenie puli słów, z których model może wybierać. Niższe wartości prowadzą do bardziej skoncentrowanego, spójnego tekstu, podczas gdy wyższe pozwalają na większą różnorodność językową.

In [8]:

# Test different temperature values
temperatures = [0.2, 0.5, 1.0, 1.5]
for temp in temperatures:
    print('------')
    print(f"\nTemperature: {temp}, Top_p: 1.0")
    generated_text = generate_text(prompt, model = model, temperature=temp, top_p=1.0)
    print(generated_text)

Device set to use mps:0


------

Temperature: 0.2, Top_p: 1.0


KeyboardInterrupt: 

Ten fragment kodu demonstruje eksperyment z wpływem temperatury na generowanie tekstu, utrzymując stałą wartość parametru top_p.

Lista `temperatures = [0.2, 0.5, 1.0, 1.5]` definiuje cztery różne wartości temperatury, które będą testowane:
- 0.2 reprezentuje bardzo niską temperaturę, gdzie model będzie bardzo konserwatywny i przewidywalny
- 0.5 to umiarkowanie niska temperatura, dająca względnie spójne, ale nie całkiem sztywne wyniki
- 1.0 to standardowa temperatura, gdzie model ma równowagę między kreatywnością a spójnością
- 1.5 to wysoka temperatura, która pozwoli modelowi na bardziej nieprzewidywalne i kreatywne generacje

Utrzymanie stałej wartości top_p = 1.0 jest istotne metodologicznie - pozwala zobaczyć czysty wpływ temperatury na generację, bez zakłóceń wynikających ze zmian innych parametrów. To jak eksperyment naukowy, gdzie zmieniamy tylko jedną zmienną, podczas gdy pozostałe pozostają stałe.

Wartość top_p = 1.0 oznacza, że model może wybierać ze wszystkich dostępnych słów, ważąc je tylko według ich prawdopodobieństwa zmodyfikowanego przez temperaturę. Jest to najbardziej "otwarte" ustawienie parametru top_p, co pozwala na pełne zaobserwowanie wpływu temperatury.

In [ ]:
# Test different top_p values
top_ps = [0.5, 0.7, 0.9, 1.0]
for top_p in top_ps:
    print('------')
    print(f"\nTemperature: 1.0, Top_p: {top_p}")
    generated_text = generate_text(prompt, model = model, temperature=1.0, top_p=top_p)
    print(generated_text)

Ten fragment kodu stanowi drugi etap eksperymentu, który bada wpływ parametru top_p na generowanie tekstu. Tym razem utrzymujemy stałą temperaturę (1.0), zmieniając wartość top_p.

Lista `top_ps = [0.5, 0.7, 0.9, 1.0]` zawiera cztery starannie dobrane wartości top_p. Parametr top_p, znany również jako próbkowanie nucleus, określa łączne prawdopodobieństwo słów, które model może wybrać w każdym kroku generacji. Przykładowo, gdy top_p = 0.7, model będzie wybierał tylko spośród słów, których skumulowane prawdopodobieństwa nie przekraczają 70% całkowitego rozkładu prawdopodobieństwa.

Wybrane wartości top_p mają następujące znaczenie:
- 0.5 oznacza bardzo selektywny wybór - model ogranicza się do najbardziej prawdopodobnych słów, co powinno prowadzić do bardziej przewidywalnego i zachowawczego tekstu
- 0.7 to umiarkowanie restrykcyjna wartość, dająca modelowi nieco więcej swobody, ale wciąż utrzymująca znaczącą kontrolę nad wyborem słów
- 0.9 pozwala na szerszy wybór słów, zbliżając się do pełnej swobody
- 1.0 daje modelowi dostęp do całego słownika, podobnie jak w poprzednim eksperymencie

Stała temperatura 1.0 została wybrana jako wartość neutralna - nie jest ani zbyt zachowawcza, ani zbyt ryzykowna. Dzięki temu możemy wyraźnie zaobserwować efekty zmiany parametru top_p.

Połączenie wyników obu eksperymentów (z różnymi temperaturami i różnymi wartościami top_p) pozwala zrozumieć, jak te dwa parametry współdziałają ze sobą w procesie generowania tekstu i jak można je dostosowywać do osiągnięcia pożądanych efektów.